# POC: Using XTable for interopability between open table formats 

Usin Xtable for transition period while migrating HUDI to delta

# Use cases

Apache XTable™ provides cross-table omni-directional interop between lakehouse table formats
Apache XTable™ is NOT a new or separate format, Apache XTable™ provides abstractions and tools for the translation of lakehouse table format metadata
Apache XTable™ is formerly known as OneTable

Apache XTable™ can be used to easily switch between any of the table formats or even benefit from more than one simultaneously. Some organizations use Apache XTable™ today because they have a diverse ecosystem of tools with polarized vendor support of table formats. Some users want lightning fast ingestion or indexing from Hudi and photon query accelerations of Delta Lake inside of Databricks. Some users want managed table services from Hudi, but also want write operations from Trino to Iceberg. Regardless of which combination of formats you need, Apache XTable™ ensures you can benefit from all 3 projects.

# Known constrains

1. Aparently HUDI CoW 1.0.0 is not working

2. Hudi and Iceberg MoR tables not supported
   
3. Delta Delete Vectors are not supported
   
4. **Synchronized transaction timestamps**
   

With Apache XTable™ you pick one primary format and one or more secondary formats. The write operations with the **primary format work as normal**. Apache XTable™ than translates the metadata from the primary format to the secondaries. When committing the metadata of the secondary formats, **the timestamp of the commit will not be the exact same timestamp as shown in the primary.**

# Before run

Make sure to compile XTable project with Java 11 and Meaven

# Working with XTable

## Setup HUDI example

Using HUDI CoW 0.15.0 version

In [1]:
spark_jar_packages = ",".join([
    "org.apache.hudi:hudi-spark3.5-bundle_2.12:0.15.0"
])

In [2]:
LOCAL_WAREHOUSE_CATALOG = "file:///home/baptvit/Documents/github/lakehouse-labs/xtable/warehouse/"

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, month, dayofmonth

spark = (
    SparkSession.builder
    .master("local[*]")
    .appName("xtable-hudi-local-playground")
    .config("spark.jars.packages", spark_jar_packages)

    # Hudi Integration
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config("spark.kryo.registrator", "org.apache.spark.HoodieSparkKryoRegistrar")
    .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.hudi.catalog.HoodieCatalog")

    # Local catalog
    .config("spark.sql.catalog.local.type", "hadoop")   # Use Hadoop catalog
    .config("spark.sql.warehouse.dir", LOCAL_WAREHOUSE_CATALOG)   # Path to store metadata
    
    .getOrCreate()
)

25/01/14 17:23:59 WARN Utils: Your hostname, baptvit resolves to a loopback address: 127.0.1.1; using 192.168.2.129 instead (on interface wlp4s0)
25/01/14 17:23:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/baptvit/.ivy2/cache
The jars for the packages stored in: /home/baptvit/.ivy2/jars
org.apache.hudi#hudi-spark3.5-bundle_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-10384895-3c75-4a93-9e9e-74d5480c1301;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.hudi#hudi-spark3.5-bundle_2.12;0.15.0 in central
	found org.apache.hive#hive-storage-api;2.8.1 in central
	found org.slf4j#slf4j-api;1.7.36 in central
:: resolution report :: resolve 125ms :: artifacts dl 4ms
	:: modules in use:
	org.apache.hive#hive-storage-api;2.8.1 from central in [default]
	org.apache.hudi#hudi-spark3.5-bundle_2.12;0.15.0 from central in [default]
	org.slf4j#slf4j-api;1.7.36 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-10384895-3c75-4a93-9e9e-74d5480c1301
	confs: [default]
	0 artifacts copied, 3

In [7]:
import random
from faker import Faker

def generate_entry(faker: Faker, country_codes: list):
    return {
        "id": faker.unique.uuid4(),
        "name":  faker.name(),
        "email": faker.email(),
        "passport": faker.passport_number(),
        "country_code": random.choice(country_codes),
        "iban": faker.iban(),
        "swift": faker.swift11(),
        "created_at": faker.past_date(start_date='-90d').strftime('%Y-%m-%d')
    }

In [8]:
def generate_dataset(num: int, seed: int):
    country_codes = ['US', 'CA', 'JP', 'KR', 'FR', 'GE', 'UK', 'BR', 'AR']
    Faker.seed(seed)
    faker = Faker()
    return [generate_entry(faker, country_codes) for _ in range(num)]

In [9]:
dataset = generate_dataset(num=100, seed=739)

In [10]:
df = spark.createDataFrame(dataset)\
        .withColumn("year", year(col("created_at")))\
        .withColumn("month", month(col("created_at")))\
        .withColumn("day", dayofmonth(col("created_at")))

In [11]:
df.count()

100

In [18]:
spark.sql("""
    CREATE DATABASE IF NOT EXISTS hudi;
""")

DataFrame[]

In [5]:
HUDI_FOLDER_PATH = "file:///home/baptvit/Documents/github/lakehouse-labs/xtable/warehouse/hudi.db/accounts"

25/01/14 17:24:16 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
df.write.format("hudi") \
    .option("hoodie.database.name", "hudi") \
    .option("hoodie.table.name", "accounts") \
    .option("hoodie.datasource.write.recordkey.field", "id") \
    .option("hoodie.datasource.write.precombine.field", "created_at") \
    .option("hoodie.datasource.write.table.type", "COPY_ON_WRITE") \
    .option("hoodie.datasource.write.operation", "upsert") \
    .option("hoodie.datasource.hive_sync.partition_fields", "year") \
    .option("hoodie.datasource.hive_sync.partition_extractor_class", "org.apache.hudi.hive.MultiPartKeysValueExtractor") \
    .option("hoodie.datasource.write.hive_style_partitioning","true") \
    .partitionBy("year") \
    .mode("append") \
    .save(HUDI_FOLDER_PATH) ## hudi needs the base full path

## Creating yaml config

In [ ]:
import yaml

# Define the YAML structure as a Python dictionary
config = {
    "sourceFormat": "HUDI",
    "targetFormats": ["DELTA", "ICEBERG"],
    "datasets": [
        {
            "tableBasePath": HUDI_FOLDER_PATH,
            "tableName": "accounts",
            "partitionSpec": "year:VALUE"
        }
    ]
}

# Write the YAML file
with open("./manifests/config.yml", "w") as yaml_file:
    yaml.dump(config, yaml_file, default_flow_style=False)

print("YAML file created successfully!")

## Convering to Delta and Iceberg

In [ ]:
!/usr/lib/jvm/java-11-openjdk-amd64/bin/java -jar /home/baptvit/Documents/github/lakehouse-labs/xtable/incubator-xtable/xtable-utilities/target/xtable-utilities_2.12-0.2.0-SNAPSHOT-bundled.jar --datasetConfig ./manifests/config.yml

## Read as Delta and Iceberg

### Read as delta table

In [ ]:
from deltalake import DeltaTable

delta_table_path = HUDI_FOLDER_PATH

delta_table = DeltaTable(delta_table_path)

df = delta_table.to_pandas()

print(df.shape)

df.head()

### Read as Iceberg table

TODO: (Reference)[https://iceberg.apache.org/docs/1.7.1/daft/] setup with minion

Need create a data catalog

In [ ]:
import polars as pl

table_path = HUDI_FOLDER_PATH + "/metadata/v2.metadata.json"
pd_lz = pl.scan_iceberg(table_path)

In [ ]:
pd_lz.count().collect()

In [ ]:
pd_lz.head().collect()

## Updating the HUDI Table

### Upsert Dataset

Editing 4 records and adding new 4 records

In [12]:
entries = [
    # Existing entries
    dataset[2], 
    dataset[4], 
    dataset[7],
    dataset[11],
    # New entries
    *generate_dataset(4, seed=1037)
]

In [13]:
for entry in entries:
    username = entry['name'].lower().replace(" ", ".")
    entry['email'] = f"{username}@domain.com"

In [14]:
upsert_df = spark.createDataFrame(entries)\
        .withColumn("year", year(col("created_at")))\
        .withColumn("month", month(col("created_at")))\
        .withColumn("day", dayofmonth(col("created_at")))

In [15]:
upsert_df.show(8, truncate=False)

+------------+----------+---------------------------+----------------------+------------------------------------+----------------+---------+-----------+----+-----+---+
|country_code|created_at|email                      |iban                  |id                                  |name            |passport |swift      |year|month|day|
+------------+----------+---------------------------+----------------------+------------------------------------+----------------+---------+-----------+----+-----+---+
|US          |2024-12-16|ann.cruz@domain.com        |GB55LFTZ50027083194346|b7e33adb-9bfe-465f-a533-1d57f8d9c9f6|Ann Cruz        |T22953641|HMAQGBCSXE8|2024|12   |16 |
|BR          |2024-12-10|cassidy.jones.md@domain.com|GB14AYNQ55188150393152|0daad7bc-25b6-4469-8a2f-2ba767f86791|Cassidy Jones MD|595954695|VTHYGBZMNOI|2024|12   |10 |
|JP          |2024-12-11|kara.thomas@domain.com     |GB02LAAF80272115976869|4cbbf121-caae-42aa-8508-3fd99bb2f762|Kara Thomas     |661814813|DULPGBWLTDU|2024|12 

In [16]:
upsert_df.write.format("hudi") \
    .option("hoodie.database.name", "hudi") \
    .option("hoodie.table.name", "accounts") \
    .option("hoodie.datasource.write.recordkey.field", "id") \
    .option("hoodie.datasource.write.precombine.field", "created_at") \
    .option("hoodie.datasource.write.table.type", "COPY_ON_WRITE") \
    .option("hoodie.datasource.write.operation", "upsert") \
    .option("hoodie.datasource.hive_sync.partition_fields", "year") \
    .option("hoodie.datasource.hive_sync.partition_extractor_class", "org.apache.hudi.hive.MultiPartKeysValueExtractor") \
    .option("hoodie.datasource.write.hive_style_partitioning","true") \
    .partitionBy("year") \
    .mode("append") \
    .save(HUDI_FOLDER_PATH) ## hudi needs the base full path

25/01/14 17:25:04 WARN DFSPropertiesConfiguration: Cannot find HUDI_CONF_DIR, please set it as the dir of hudi-defaults.conf
25/01/14 17:25:04 WARN DFSPropertiesConfiguration: Properties file file:/etc/hudi/conf/hudi-defaults.conf not found. Ignoring to load props file


# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]


25/01/14 17:25:05 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-hbase.properties,hadoop-metrics2.properties
25/01/14 17:25:07 WARN HoodieSparkSqlWriterInternal: Closing write client       


In [21]:
spark.sql(
    f"""
    CREATE TABLE IF NOT EXISTS hudi.accounts
        USING hudi
        OPTIONS (
          path = '{HUDI_FOLDER_PATH}'
        );
    """
)

DataFrame[]

In [22]:
spark.sql("""
    call show_commits (
        table => 'hudi.accounts',
        from_commit => '2'
    )    
""").show(vertical=True, truncate=False)

-RECORD 0-----------------------------------------
 commit_time                  | 20250114172504047 
 state_transition_time        | 20250114172507314 
 action                       | commit            
 total_bytes_written          | 885548            
 total_files_added            | 0                 
 total_files_updated          | 2                 
 total_partitions_written     | 2                 
 total_records_written        | 104               
 total_update_records_written | 4                 
 total_errors                 | 0                 
-RECORD 1-----------------------------------------
 commit_time                  | 20250114162008677 
 state_transition_time        | 20250114162011988 
 action                       | commit            
 total_bytes_written          | 885144            
 total_files_added            | 2                 
 total_files_updated          | 0                 
 total_partitions_written     | 2                 
 total_records_written        |

## Read as Delta and Iceberg without sync

### Read as delta table

In [23]:
from deltalake import DeltaTable

delta_table_path = HUDI_FOLDER_PATH

delta_table = DeltaTable(delta_table_path)

df = delta_table.to_pandas()

print(df.shape)

df.head()

(100, 16)


,_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,country_code,created_at,email,iban,id,name,passport,swift,year,month,day
0,20250114162008677,20250114162008677_1_0,8da2d590-9035-4f97-9064-e4508ffc47d6,year=2024,2744bc50-8dfa-42fa-b547-da83e2f96038-0_1-26-14...,JP,2024-12-10,cgarcia@example.org,GB56KBQA28505176292677,8da2d590-9035-4f97-9064-e4508ffc47d6,John Morris,G04565275,ENVXGBFG8LH,2024,12,10
1,20250114162008677,20250114162008677_1_1,ee99b01d-ee7e-45b6-ae7d-954a4fc810fa,year=2024,2744bc50-8dfa-42fa-b547-da83e2f96038-0_1-26-14...,CA,2024-12-03,heathstephanie@example.net,GB06KGXT72498366139972,ee99b01d-ee7e-45b6-ae7d-954a4fc810fa,Lisa Krueger,W64138100,WZHEGB02AH6,2024,12,3
2,20250114162008677,20250114162008677_1_2,d8797d55-a4cd-4354-a7ae-f7454a82bde8,year=2024,2744bc50-8dfa-42fa-b547-da83e2f96038-0_1-26-14...,AR,2024-10-28,mwinters@example.com,GB30DUBO07081102663473,d8797d55-a4cd-4354-a7ae-f7454a82bde8,Lisa Murray,A07022327,GXWXGBLYODL,2024,10,28
3,20250114162008677,20250114162008677_1_3,019aea9a-35bf-4a7c-be65-a783924affd0,year=2024,2744bc50-8dfa-42fa-b547-da83e2f96038-0_1-26-14...,UK,2024-11-10,robertlynch@example.com,GB72XSCN85943044334322,019aea9a-35bf-4a7c-be65-a783924affd0,Christine Crawford,G48667446,YWABGB5TK1G,2024,11,10
4,20250114162008677,20250114162008677_1_4,757daf25-ddd6-410d-874b-f13836813553,year=2024,2744bc50-8dfa-42fa-b547-da83e2f96038-0_1-26-14...,UK,2024-12-28,seanhenderson@example.org,GB96HUZF24175067188304,757daf25-ddd6-410d-874b-f13836813553,Sherri Hickman,529955944,OXTEGBB860C,2024,12,28


### Read as Iceberg table

In [24]:
import polars as pl

table_path = HUDI_FOLDER_PATH + "/metadata/v2.metadata.json"
pd_lz = pl.scan_iceberg(table_path)

In [25]:
pd_lz.count().collect()

_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,country_code,created_at,email,iban,id,name,passport,swift,year,month,day
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
100,100,100,100,100,100,100,100,100,100,100,100,100,100,100,100


In [26]:
pd_lz.head().collect()

_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,country_code,created_at,email,iban,id,name,passport,swift,year,month,day
str,str,str,str,str,str,str,str,str,str,str,str,str,i32,i32,i32
"""20250114162008677""","""20250114162008677_1_0""","""8da2d590-9035-4f97-9064-e4508f…","""year=2024""","""2744bc50-8dfa-42fa-b547-da83e2…","""JP""","""2024-12-10""","""cgarcia@example.org""","""GB56KBQA28505176292677""","""8da2d590-9035-4f97-9064-e4508f…","""John Morris""","""G04565275""","""ENVXGBFG8LH""",2024,12,10
"""20250114162008677""","""20250114162008677_1_1""","""ee99b01d-ee7e-45b6-ae7d-954a4f…","""year=2024""","""2744bc50-8dfa-42fa-b547-da83e2…","""CA""","""2024-12-03""","""heathstephanie@example.net""","""GB06KGXT72498366139972""","""ee99b01d-ee7e-45b6-ae7d-954a4f…","""Lisa Krueger""","""W64138100""","""WZHEGB02AH6""",2024,12,3
"""20250114162008677""","""20250114162008677_1_2""","""d8797d55-a4cd-4354-a7ae-f7454a…","""year=2024""","""2744bc50-8dfa-42fa-b547-da83e2…","""AR""","""2024-10-28""","""mwinters@example.com""","""GB30DUBO07081102663473""","""d8797d55-a4cd-4354-a7ae-f7454a…","""Lisa Murray""","""A07022327""","""GXWXGBLYODL""",2024,10,28
"""20250114162008677""","""20250114162008677_1_3""","""019aea9a-35bf-4a7c-be65-a78392…","""year=2024""","""2744bc50-8dfa-42fa-b547-da83e2…","""UK""","""2024-11-10""","""robertlynch@example.com""","""GB72XSCN85943044334322""","""019aea9a-35bf-4a7c-be65-a78392…","""Christine Crawford""","""G48667446""","""YWABGB5TK1G""",2024,11,10
"""20250114162008677""","""20250114162008677_1_4""","""757daf25-ddd6-410d-874b-f13836…","""year=2024""","""2744bc50-8dfa-42fa-b547-da83e2…","""UK""","""2024-12-28""","""seanhenderson@example.org""","""GB96HUZF24175067188304""","""757daf25-ddd6-410d-874b-f13836…","""Sherri Hickman""","""529955944""","""OXTEGBB860C""",2024,12,28


## Sync to Delta and Iceberg

In [27]:
!/usr/lib/jvm/java-11-openjdk-amd64/bin/java -jar /home/baptvit/Documents/github/lakehouse-labs/xtable/incubator-xtable/xtable-utilities/target/xtable-utilities_2.12-0.2.0-SNAPSHOT-bundled.jar --datasetConfig ./manifests/config.yml

2025-01-14 17:27:45 INFO  org.apache.xtable.utilities.RunSync:149 - Running sync for basePath file:///home/baptvit/Documents/github/lakehouse-labs/xtable/warehouse/hudi.db/accounts for following table formats [DELTA, ICEBERG]
2025-01-14 17:27:45 INFO  org.apache.hudi.common.table.HoodieTableMetaClient:133 - Loading HoodieTableMetaClient from file:///home/baptvit/Documents/github/lakehouse-labs/xtable/warehouse/hudi.db/accounts
2025-01-14 17:27:45 WARN  org.apache.hadoop.util.NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2025-01-14 17:27:45 INFO  org.apache.hudi.common.table.HoodieTableConfig:276 - Loading table properties from file:/home/baptvit/Documents/github/lakehouse-labs/xtable/warehouse/hudi.db/accounts/.hoodie/hoodie.properties
2025-01-14 17:27:45 INFO  org.apache.hudi.common.table.HoodieTableMetaClient:152 - Finished Loading Table of type COPY_ON_WRITE(version=1, baseFileFormat=PARQUET) from file:///

## Read as Delta and Iceberg after sync

### Read as delta table

In [28]:
from deltalake import DeltaTable

delta_table_path = HUDI_FOLDER_PATH

delta_table = DeltaTable(delta_table_path)

df = delta_table.to_pandas()

print(df.shape)

df.head()

(104, 16)


,_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,country_code,created_at,email,iban,id,name,passport,swift,year,month,day
0,20250114162008677,20250114162008677_0_0,c7b1cee6-9ec9-4455-abb4-b032793d873e,year=2025,66945abe-8145-4e86-bcc4-4b015e28532e-0_0-20-15...,FR,2025-01-11,melissa80@example.net,GB76KDUB00543518056217,c7b1cee6-9ec9-4455-abb4-b032793d873e,Jeanette Green,R72046482,JGIDGBRWHD4,2025,1,11
1,20250114162008677,20250114162008677_0_1,524da8f0-b784-4638-ae01-a7b7d13607de,year=2025,66945abe-8145-4e86-bcc4-4b015e28532e-0_0-20-15...,FR,2025-01-09,robertmcintyre@example.org,GB81OZIU60490488184758,524da8f0-b784-4638-ae01-a7b7d13607de,Kim Matthews,T89100458,DYFAGBTAB1H,2025,1,9
2,20250114162008677,20250114162008677_0_2,3cc19a63-9307-4482-b775-bf84bb7f524a,year=2025,66945abe-8145-4e86-bcc4-4b015e28532e-0_0-20-15...,AR,2025-01-03,ltaylor@example.net,GB33IROQ95211505041890,3cc19a63-9307-4482-b775-bf84bb7f524a,Kaitlyn Taylor,207116683,BTTSGBIWF66,2025,1,3
3,20250114162008677,20250114162008677_0_3,20ad946c-ea9b-4f01-8eaa-504a06b92fa0,year=2025,66945abe-8145-4e86-bcc4-4b015e28532e-0_0-20-15...,CA,2025-01-05,smithalexandria@example.org,GB77IYXH23843855501257,20ad946c-ea9b-4f01-8eaa-504a06b92fa0,Ashlee Aguilar,E06363228,WSGWGBWHXSB,2025,1,5
4,20250114162008677,20250114162008677_0_4,c2769b84-54d4-476c-b27c-856672cbbd34,year=2025,66945abe-8145-4e86-bcc4-4b015e28532e-0_0-20-15...,UK,2025-01-10,christianbrian@example.org,GB30IFVT02678036529533,c2769b84-54d4-476c-b27c-856672cbbd34,Jonathan Weber,897841773,CGPVGB002QU,2025,1,10


### Read as Iceberg table

In [32]:
import polars as pl

table_path = HUDI_FOLDER_PATH + "/metadata/v3.metadata.json"
pd_lz = pl.scan_iceberg(table_path)

In [33]:
pd_lz.count().collect()

_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,country_code,created_at,email,iban,id,name,passport,swift,year,month,day
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
104,104,104,104,104,104,104,104,104,104,104,104,104,104,104,104


In [34]:
pd_lz.head().collect()

_hoodie_commit_time,_hoodie_commit_seqno,_hoodie_record_key,_hoodie_partition_path,_hoodie_file_name,country_code,created_at,email,iban,id,name,passport,swift,year,month,day
str,str,str,str,str,str,str,str,str,str,str,str,str,i32,i32,i32
"""20250114162008677""","""20250114162008677_0_0""","""c7b1cee6-9ec9-4455-abb4-b03279…","""year=2025""","""66945abe-8145-4e86-bcc4-4b015e…","""FR""","""2025-01-11""","""melissa80@example.net""","""GB76KDUB00543518056217""","""c7b1cee6-9ec9-4455-abb4-b03279…","""Jeanette Green""","""R72046482""","""JGIDGBRWHD4""",2025,1,11
"""20250114162008677""","""20250114162008677_0_1""","""524da8f0-b784-4638-ae01-a7b7d1…","""year=2025""","""66945abe-8145-4e86-bcc4-4b015e…","""FR""","""2025-01-09""","""robertmcintyre@example.org""","""GB81OZIU60490488184758""","""524da8f0-b784-4638-ae01-a7b7d1…","""Kim Matthews""","""T89100458""","""DYFAGBTAB1H""",2025,1,9
"""20250114162008677""","""20250114162008677_0_2""","""3cc19a63-9307-4482-b775-bf84bb…","""year=2025""","""66945abe-8145-4e86-bcc4-4b015e…","""AR""","""2025-01-03""","""ltaylor@example.net""","""GB33IROQ95211505041890""","""3cc19a63-9307-4482-b775-bf84bb…","""Kaitlyn Taylor""","""207116683""","""BTTSGBIWF66""",2025,1,3
"""20250114162008677""","""20250114162008677_0_3""","""20ad946c-ea9b-4f01-8eaa-504a06…","""year=2025""","""66945abe-8145-4e86-bcc4-4b015e…","""CA""","""2025-01-05""","""smithalexandria@example.org""","""GB77IYXH23843855501257""","""20ad946c-ea9b-4f01-8eaa-504a06…","""Ashlee Aguilar""","""E06363228""","""WSGWGBWHXSB""",2025,1,5
"""20250114162008677""","""20250114162008677_0_4""","""c2769b84-54d4-476c-b27c-856672…","""year=2025""","""66945abe-8145-4e86-bcc4-4b015e…","""UK""","""2025-01-10""","""christianbrian@example.org""","""GB30IFVT02678036529533""","""c2769b84-54d4-476c-b27c-856672…","""Jonathan Weber""","""897841773""","""CGPVGB002QU""",2025,1,10
